In [1]:
%cd ../..

/home/work/AnacondaProjects/combinators


In [2]:
import logging

import numpy as np
import probtorch
import torch

from examples.fep_control import fep_control
from combinators.model import active
from combinators.model import compose, foldable
from combinators.inference import importance
from combinators import utils

In [3]:
logging.basicConfig(format='%(asctime)s %(message)s', datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)

In [4]:
target = fep_control.MountainCarStep(state_dim=2, batch_shape=(1,), trainable=False)
proposal = fep_control.RecognitionStep(state_dim=2, observation_dim=2, batch_shape=(1,), discrete_actions=False, name='MountainCarStep', trainable=True)
agent = importance.propose(target, proposal)

In [5]:
theta, graph, log_weight = active.active_inference_test(agent, 'MountainCarContinuous-v0', online_inference=False, iterations=1000)

In [6]:
theta, graph, log_weight = active.active_inference(agent, 'MountainCarContinuous-v0', lr=1e-2, episodes=10, episode_length=1000, dream=False)

05/30/2019 22:40:03 ELBO=-4.63767539e+04 at episode 0 of length 78
05/30/2019 22:40:14 ELBO=-7.33113916e+03 at episode 1 of length 1000
05/30/2019 22:40:26 ELBO=-6.45362500e+03 at episode 2 of length 1000
05/30/2019 22:40:40 ELBO=-8.02626807e+03 at episode 3 of length 163
05/30/2019 22:40:52 ELBO=-6.35342139e+03 at episode 4 of length 439
05/30/2019 22:41:02 ELBO=-5.49975781e+03 at episode 5 of length 299
05/30/2019 22:41:11 ELBO=-5.33544336e+03 at episode 6 of length 168
05/30/2019 22:41:21 ELBO=-9.15691602e+03 at episode 7 of length 585
05/30/2019 22:41:32 ELBO=-6.21992188e+03 at episode 8 of length 637
05/30/2019 22:41:42 ELBO=-6.13329053e+03 at episode 9 of length 237


In [7]:
log_weight

tensor([-6133.2905], grad_fn=<AddBackward0>)

In [16]:
theta, graph, log_weight = active.active_inference_test(agent, 'MountainCarContinuous-v0', online_inference=False, iterations=1000)

In [9]:
theta

(tensor([[-0.1900,  1.7639]], grad_fn=<AddBackward0>),
 tensor([[0.2019]], grad_fn=<AddBackward0>))

In [10]:
log_weight

tensor([-558.7200], grad_fn=<AddBackward0>)

In [11]:
torch.save(agent, 'examples/fep_control/fep_mountain_car_agent.dat')